In [1]:
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Set paths
project_root = '/content/drive/MyDrive/MLAI/Multimodal sentiment analysis/MVSA_Single'
data_dir = os.path.join(project_root, 'data')
label_file = os.path.join(project_root, 'label.txt')


Mounted at /content/drive


In [2]:
# Read the file
import pandas as pd

df = pd.read_csv(label_file, sep="\t")

# Split the combined sentiment column
df[['text_sentiment', 'image_sentiment']] = df['text,image'].str.split(',', expand=True)

# Drop the original column
df.drop(columns=['text,image'], inplace=True)
print(df.shape)
df.head()

(4869, 3)


,ID,text_sentiment,image_sentiment
0,1,neutral,positive
1,2,neutral,positive
2,3,neutral,positive
3,4,positive,positive
4,5,positive,positive


In [3]:
from transformers import BertTokenizer
import tensorflow as tf
from tensorflow.keras.preprocessing import image as keras_image

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

IMG_SIZE = 224
MAX_LEN = 30
BATCH_SIZE = 32

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [4]:
from sklearn.model_selection import train_test_split

# Only keep rows where both image and text files exist
df['image_path'] = df['ID'].apply(lambda x: os.path.join(data_dir, f"{x}.jpg"))
df['text_path'] = df['ID'].apply(lambda x: os.path.join(data_dir, f"{x}.txt"))

df = df[df['image_path'].apply(os.path.exists) & df['text_path'].apply(os.path.exists)]

# Use text sentiment labels (or image sentiment if you want)
df['label'] = df['text_sentiment']  # you can switch to 'image_sentiment' if needed

# Encode sentiment to integer
label_map = {'positive': 0, 'neutral': 1, 'negative': 2}
df['label'] = df['text_sentiment'].map(label_map)
df = df.dropna()

# Split into train and validation sets
from sklearn.model_selection import train_test_split
train_df, vali_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)



<ipython-input-4-ac73862ccadc>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['text_sentiment']  # you can switch to 'image_sentiment' if needed
<ipython-input-4-ac73862ccadc>:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'] = df['text_sentiment'].map(label_map)


In [5]:
def load_text(text_path):
    with open(text_path, 'r', encoding='utf-8', errors='ignore') as f:
        return f.read().strip()


def preprocess(image_path, text_path, label):
    # Process image
    img = keras_image.load_img(image_path, target_size=(IMG_SIZE, IMG_SIZE))
    img = keras_image.img_to_array(img) / 255.0  # Normalize
    img = tf.convert_to_tensor(img, dtype=tf.float32)

    # Process text
    text = load_text(text_path)
    tokens = tokenizer(text, max_length=MAX_LEN, padding='max_length', truncation=True, return_tensors='tf')

    input_ids = tf.squeeze(tokens['input_ids'])
    attention_mask = tf.squeeze(tokens['attention_mask'])

    # Convert label to one-hot
    label = tf.one_hot(label, depth=3)

    return img, input_ids, attention_mask, label


In [6]:
def make_tf_dataset(df):
    paths = list(zip(df['image_path'], df['text_path'], df['label']))

    def gen():
        for img_path, txt_path, label in paths:
            yield preprocess(img_path, txt_path, label)

    dataset = tf.data.Dataset.from_generator(
        gen,
        output_signature=(
            tf.TensorSpec(shape=(IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(MAX_LEN,), dtype=tf.int32),
            tf.TensorSpec(shape=(MAX_LEN,), dtype=tf.int32),
            tf.TensorSpec(shape=(3,), dtype=tf.float32),
        )
    )

    return dataset.shuffle(500).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

# Create the datasets
train_dataset = make_tf_dataset(train_df)
vali_dataset = make_tf_dataset(vali_df)


In [ ]:
# Check one batch from the training dataset
for images, input_ids, attention_masks, labels in train_dataset.take(1):
    print("Train Dataset:")
    print("Images shape:", images.shape)
    print("Input IDs shape:", input_ids.shape)
    print("Attention Masks shape:", attention_masks.shape)
    print("Labels shape:", labels.shape)
    break

# Check one batch from the validation dataset
for images, input_ids, attention_masks, labels in vali_dataset.take(1):
    print("\nValidation Dataset:")
    print("Images shape:", images.shape)
    print("Input IDs shape:", input_ids.shape)
    print("Attention Masks shape:", attention_masks.shape)
    print("Labels shape:", labels.shape)
    break


In [3]:
import os
import pandas as pd
from sklearn.model_selection import train_test_split
from transformers import BertTokenizer
import tensorflow as tf
from tensorflow.keras.preprocessing import image as keras_image

# Paths
project_root = '/content/drive/MyDrive/MLAI/Multimodal sentiment analysis/MVSA_Single'
data_dir = os.path.join(project_root, 'data')
label_file = os.path.join(project_root, 'label.txt')

# Read the file
import pandas as pd
df = pd.read_csv(label_file, sep="\t")
# Split the combined sentiment column
df[['text_sentiment', 'image_sentiment']] = df['text,image'].str.split(',', expand=True)
# Drop the original column
df.drop(columns=['text,image'], inplace=True)
print(df.shape)
df.head()

# Load labels into dataframe (adjust separator if needed)
df = pd.read_csv(label_file, sep=',', header=None, names=['id', 'text_sentiment', 'image_sentiment'])

# Add full file paths
df['image_path'] = df['id'].apply(lambda x: os.path.join(data_dir, f"{x}.jpg"))
df['text_path'] = df['id'].apply(lambda x: os.path.join(data_dir, f"{x}.txt"))

# Filter out rows where either image or text file doesn't exist
df = df[df['image_path'].apply(os.path.exists) & df['text_path'].apply(os.path.exists)].copy()

# Map sentiment labels to integers
label_map = {'positive': 0, 'neutral': 1, 'negative': 2}
df['label'] = df['text_sentiment'].map(label_map)
df = df.dropna(subset=['label']).reset_index(drop=True)

# Tokenizer and constants
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
IMG_SIZE = 224
MAX_LEN = 30
BATCH_SIZE = 32

def load_text_safe(text_path):
    try:
        with open(text_path, 'r', encoding='utf-8') as f:
            return f.read().strip()
    except UnicodeDecodeError as e:
        print(f"[UnicodeDecodeError] Skipping file: {text_path} | Error: {e}")
        return None  # Skip bad text files

def preprocess(img_path, text_path, label):
    text = load_text_safe(text_path)
    if text is None:
        return None  # Will filter out later

    # Image preprocessing
    img = keras_image.load_img(img_path, target_size=(IMG_SIZE, IMG_SIZE))
    img = keras_image.img_to_array(img) / 255.0
    img = tf.convert_to_tensor(img, dtype=tf.float32)

    # Text tokenization
    tokens = tokenizer(text, max_length=MAX_LEN, padding='max_length', truncation=True, return_tensors='tf')
    input_ids = tf.squeeze(tokens['input_ids'])
    attention_mask = tf.squeeze(tokens['attention_mask'])

    label = tf.one_hot(label, depth=3)
    return img, input_ids, attention_mask, label

# Generator filtering out invalid text
def gen(df):
    for _, row in df.iterrows():
        processed = preprocess(row['image_path'], row['text_path'], row['label'])
        if processed is not None:
            yield processed

def make_dataset(df):
    output_signature = (
        tf.TensorSpec(shape=(IMG_SIZE, IMG_SIZE, 3), dtype=tf.float32),
        tf.TensorSpec(shape=(MAX_LEN,), dtype=tf.int32),
        tf.TensorSpec(shape=(MAX_LEN,), dtype=tf.int32),
        tf.TensorSpec(shape=(3,), dtype=tf.float32),
    )

    dataset = tf.data.Dataset.from_generator(lambda: gen(df), output_signature=output_signature)
    dataset = dataset.shuffle(500).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    return dataset

# Split train/val
train_df, val_df = train_test_split(df, test_size=0.2, stratify=df['label'], random_state=42)

# Create datasets
train_dataset = make_dataset(train_df)
val_dataset = make_dataset(val_df)

print(f"Train samples: {len(train_df)}")
print(f"Validation samples: {len(val_df)}")


(4869, 3)


ValueError: With n_samples=0, test_size=0.2 and train_size=None, the resulting train set will be empty. Adjust any of the aforementioned parameters.